In [1]:
from jupyter_plotly_dash import JupyterDash
from bson.objectid import ObjectId
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
from crud import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = 'accuser'
password = 'password'

shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))

                               
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'GraziosoSalvareSmall.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H2("Developed by Lukas Mueller (2022)"))),
    html.Hr(),
    
    #this row houses four radio buttons that are used to filter the data
    html.Div(className='row',
        style={'display': 'flex'},
            children=[
                dcc.RadioItems(
                    id="filter-type",
                    #Water is short for Water Rescue, Wilderness is short for Mountain or Wilderness Rescue,
                    #Disaster is short for Disaster Rescue or Individual Tracking, Reset clears all filters
                    options=[
                       {'label': 'Water', 'value': 'water'},
                       {'label': 'Wilderness', 'value': 'wild'},
                       {'label': 'Disaster', 'value': 'disaster'},
                       {'label': 'Reset', 'value': 'reset'}
                    ],
                    value='reset'
                )
            ]
    ),

    html.Hr(),
    #this is the layout for the data table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #these are options which mostly provide for native interactivity with the table
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="False",
        row_selectable="False",
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,

        
        #Below is some simple styling for the data table to make it easier to look at...
        
        #table striping implemented to make visual tracking of a document's data easier
        style_cell_conditional=[
            {
            'textAlign': 'right',
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(220, 220, 220)',
            }
        ],
        #assertion of text color
        style_data={
            'color': 'black',

        },
        #styling for the table header
        style_header={
            'backgroundColor': 'rgb(210, 210, 210)',
            'color': 'black',
            'fontWeight': 'bold'
        },
        #sizing for table cells, to increase presentability based on length of data
        style_cell={
            'minHeight': '16px', 'height': '16px', 'maxHeight': '16px',
            'minWidth': '160px', 'width': '160px', 'maxWidth': '160px',
            'whiteSpace': 'normal'
        },
    ),
    
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side 
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 #this is the pie chart that is used
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    ),
                 #this is the geolocation map that is used
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
             ])
])
                               
                

#############################################
# Interaction Between Components / Controller
#############################################

#Radio filter callback, with different options to filter for water rescue, mountain and wilderness, 
#and disaster and tracking canine candidates. Reset filter removes other filters to display all data.
@app.callback(
    [Output('datatable-id','data'), 
    Output('datatable-id','columns')],
    [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    #provides the search query for dogs well suited for Water Rescue 
    if filter_type == 'water':
        df = pd.DataFrame.from_records(shelter.read_all({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix",
                              "Chesapeake Bay Retriever", 
                              "Newfoundland"
                             ]
                     },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))

    #provides the search query for dogs well suited for Mountain or Wilderness rescue
    elif filter_type == 'wild':
        df = pd.DataFrame.from_records(shelter.read_all({
            "animal_type" : "Dog",
            "breed" : {"$in": ["German Shepard","Alaskan Malamute",
                              "Old English Sheepdog", "Siberian Husky", 
                              "Rottweiler"
                             ]
                     },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))

    #provides the search query for dogs well suited for Disaster Rescue or Individual Tracking
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(shelter.read_all({
            "animal_type": "Dog",
            "breed" : {"$in": ["Doberman Pinscher","German Shepard",
                              "Golden Retriever", "Bloodhound",
                              "Rottweiler"
                             ]
                     },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))

    # Reset query filter by querying all the data without specifying any particulars
    elif filter_type == 'reset':
        df = pd.DataFrame.from_records(shelter.read_all({}))
    
    #even though this would be logically "impossible"...
    else: 
        return 0
    
    #store the columns and the queried data in the columns and data variable, and return to enable display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)

#Highlight any column that is selected via checkboxes displayed in header cells
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_style_c(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#populate and construct pie chart graph with data returned from rescue type data filtering, here based on breed 
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)    
    fig = px.pie(dff, names ="breed")
    pieChart = [
        dcc.Graph(figure=fig)
    ]
    return pieChart

#use geolocation data associated with a record to display location, animal name,
#in response to the selection of a table row data
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    #identifies and stores the row_id of the last row that was selected, used to get geolocation coordinates
    last_selected = row_ids[len(row_ids) - 1]
    
    #use default data on load, but needs to be modified to work as intended in future iterations
    # Austin TX is at [30.75,-97.48]
    if last_selected is None:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, 
                   center=[30.75,-97.48], 
                   zoom=10,
                   children=[
                       dl.TileLayer(id='base-layer-id'),
                       # Map marker with tool tip and popup
                       dl.Marker(position=[30.75,-97.48], 
                                 children=[
                                     #column 4 gives the breed, default is for row 1 animal.
                                     #this causes the breed to appear in map display on hover
                                     dl.Tooltip(1,4),
                                     #animal name pops up when geolocation locus is selected
                                     dl.Popup([
                                         html.H1('Animal Name'),
                                         #column 9 gives the animal's name, default is for row 1 animal
                                         html.P(1,9)
                                     ])
                                 ])
                   ])
        ]
    #in the case where a row has been selected
    else:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, 
                   #last_selected is the selected row, column 13 gives the latitude, colum 14 gives longitude
                   #the map loads with the corresponding coordinates at it center
                   center=[dff.iloc[last_selected,13], dff.iloc[last_selected ,14]], 
                   zoom=10,
                   children=[
                       dl.TileLayer(id='base-layer-id'),
                       # Marker with tool tip and popup
                       dl.Marker(position=[dff.iloc[last_selected,13], dff.iloc[last_selected,14]], 
                                 children=[
                                     #column 4 of the selected row gives the breed
                                     dl.Tooltip(dff.iloc[last_selected,4]),
                                     dl.Popup([
                                         html.H1('Animal Name'),
                                         #column 9 of the selected row gives the animal's name
                                         html.P(dff.iloc[last_selected,9])
                                     ])
                                 ])
                   ])
        ]


app